In [17]:
from langchain.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="bolt+s://b4125c9b5766887e8fc1f2a0217adc72.neo4jsandbox.com:7687",
    username="neo4j",
    password="bells-drips-decks",
)

r = graph.query("MATCH (m:Movie{title: 'Toy Story'}) RETURN m")
print(r)

ValueError: Could not connect to Neo4j database. Please ensure that the url is correct

# Agents


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool

llm = ChatOpenAI(
    openai_api_key="sk-..."
)

prompt = PromptTemplate(
    template="""
    You are a movie expert. You find movies from a genre or plot.

    ChatHistory:{chat_history}
    Question:{input}
    """,
    input_variables=["chat_history", "input"]
    )

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chat_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

tools = [
    Tool.from_function(
        name="ChatOpenAI",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=chat_chain.run,
        return_direct=True
    )
]

agent = initialize_agent(
    tools, llm, memory=memory,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    max_iterations=3,
    verbose=True,
    handle_parsing_errors=True,
)

"""
- max_iterations - the maximum number of iterations to run the LLM for. This is useful in preventing the LLM from running for too long or entering an infinite loop.

- verbose - if True the agent will print out the LLM output and the tool output.

- handle_parsing_errors - if True the agent will handle parsing errors and return a message to the user.
"""


while True:
    q = input(">")
    print(agent.run(q))

# LLM & Neo4j course

In [8]:
from langchain.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="bolt://3.93.219.226:7687",
    username="neo4j",
    password="ticks-sex-fields"
)

r = graph.query("MATCH (m:Movie{title: 'Toy Story'}) RETURN m")
print(r)

Failed to write data to connection IPv4Address(('44.200.7.27', 7687)) (ResolvedIPv4Address(('44.200.7.27', 7687)))


[{'m': {'languages': ['English'], 'year': 1995, 'imdbId': '0114709', 'runtime': 81, 'imdbRating': 8.3, 'movieId': '1', 'countries': ['USA'], 'imdbVotes': 591836, 'title': 'Toy Story', 'url': 'https://themoviedb.org/movie/862', 'revenue': 373554033, 'tmdbId': '862', 'plot': "A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room.", 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg', 'released': '1995-11-22', 'budget': 30000000}}]


In [9]:
print(graph.schema)


        Node properties are the following:
        [{'labels': 'Movie', 'properties': [{'property': 'url', 'type': 'STRING'}, {'property': 'runtime', 'type': 'INTEGER'}, {'property': 'revenue', 'type': 'INTEGER'}, {'property': 'budget', 'type': 'INTEGER'}, {'property': 'imdbRating', 'type': 'FLOAT'}, {'property': 'released', 'type': 'STRING'}, {'property': 'countries', 'type': 'LIST'}, {'property': 'languages', 'type': 'LIST'}, {'property': 'plot', 'type': 'STRING'}, {'property': 'imdbVotes', 'type': 'INTEGER'}, {'property': 'imdbId', 'type': 'STRING'}, {'property': 'year', 'type': 'INTEGER'}, {'property': 'poster', 'type': 'STRING'}, {'property': 'movieId', 'type': 'STRING'}, {'property': 'tmdbId', 'type': 'STRING'}, {'property': 'title', 'type': 'STRING'}]}, {'labels': 'Genre', 'properties': [{'property': 'name', 'type': 'STRING'}]}, {'labels': 'User', 'properties': [{'property': 'userId', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}]}, {'labels': 'Actor', 'properties':

Querying a vector index

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector

embedding_provider = OpenAIEmbeddings(openai_api_key="sk-...")

movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    url="bolt://3.93.219.226:7687",
    username="neo4j",
    password="ticks-sex-fields",
    index_name="moviePlots",
    embedding_node_property="embedding",
    text_node_property="plot",
)

r = movie_plot_vector.similarity_search("A movie where aliens land and attack earth.")
print(r)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector

embedding_provider = OpenAIEmbeddings()

documents = ["Text to be indexed"]

new_vector = Neo4jVector.from_documents(
    documents,
    embedding_provider,
    url="bolt://3.93.219.226:7687",
    username="neo4j",
    password="ticks-sex-fields",
    index_name="myVectorIndex",
    node_label="Chunk",
    text_node_property="text",
    embedding_node_property="embedding",
    create_id_index=True,
)

The following code would create embeddings and a new index called myVectorIndex in the database for Chunk nodes with a text property:

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector

embedding_provider = OpenAIEmbeddings()

documents = ["Text to be indexed"]

new_vector = Neo4jVector.from_documents(
    documents,
    embedding_provider,
    url="bolt://3.93.219.226:7687",
    username="neo4j",
    password="ticks-sex-fields",
    index_name="myVectorIndex",
    node_label="Chunk",
    text_node_property="text",
    embedding_node_property="embedding",
    create_id_index=True,
)

In [13]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, RetrievalQA
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool, YouTubeSearchTool
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector
import streamlit as st

OPENAI_API_KEY = st.secrets["OPENAI_API_KEY"]

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY
)

youtube = YouTubeSearchTool()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

prompt = PromptTemplate(
    template="""
    You are a movie expert. You find movies from a genre or plot.

    ChatHistory:{chat_history}
    Question:{input}
    """,
    input_variables=["chat_history", "input"]
    )

chat_chain = LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True)

embedding_provider = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    url="bolt://3.93.219.226:7687",
    username="neo4j",
    password="ticks-sex-fields",
    index_name="moviePlots",
    embedding_node_property="embedding",
    text_node_property="plot",
)

retrievalQA = RetrievalQA.from_llm(
    llm=llm,
    retriever=movie_plot_vector.as_retriever(),
    verbose=True,
    return_source_documents=True
)

def run_retriever(query):
    results = retrievalQA({"query":query})
    return str(results)

tools = [
    Tool.from_function(
        name="ChatOpenAI",
        description="For when you need to chat about movies, genres or plots. The question will be a string. Return a string.",
        func=chat_chain.run,
        return_direct=True
    ),
    Tool.from_function(
        name="YouTubeSearchTool",
        description="For when you need a link to a movie trailer. The question will be a string. Return a link to a YouTube video.",
        func=youtube.run,
        return_direct=True
    ),
    Tool.from_function(
        name="PlotRetrieval",
        description="For when you need to compare a plot to a movie. The question will be a string. Return a string.",
        func=run_retriever,
        return_direct=True
    )
]

agent = initialize_agent(
    tools, llm, memory=memory,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)

while True:
    q = input(">")
    print(agent.run(q))

ValueError: Could not connect to Neo4j database. Please ensure that the url is correct

When the program runs, the RetrievalQA chain will use the movie_plot_vector retriever to retrieve documents from the moviePlots index and pass them to the chat_llm language model.

## GraphCypherQAChain 

In [ ]:
st.secrets["OPENAI_API_KEY"]

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
import streamlit as st

llm = ChatOpenAI(
    openai_api_key=st.secrets["OPENAI_API_KEY"]
)

graph = Neo4jGraph(
    url="bolt://3.93.219.226:7687",
    username="neo4j",
    password="ticks-sex-fields",
)

CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Schema: {schema}
Question: {question}
"""

#The "schema" will be automatically generated from the graph database and passed to the LLM. 
# The "question" will be the user’s question.

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True # see the cypher query
)

cypher_chain.run("What role did Tom Hanks play in Toy Story?")

ValueError: Could not connect to Neo4j database. Please ensure that the url is correct

The LLM used the database schema to generate an appropriate Cypher query. Langchain then executed the query against the graph database, and the result returned.